In [21]:
# Create submission for 2025 by using the receivals from 2024
import pandas as pd

orders_with_receivals = pd.read_csv("../data_cleaned/orders_with_receivals_detailed.csv", parse_dates=["delivery_date", "created_date_time", "modified_date_time", "date_arrival"])
orders_with_receivals_2024 = orders_with_receivals[orders_with_receivals['date_arrival'].dt.year == 2024]
orders_with_receivals_2024 = orders_with_receivals_2024[['rm_id', 'date_arrival', 'net_weight']]

In [22]:
sample_submission = pd.read_csv("../data/sample_submission.csv")
prediction_mapping = pd.read_csv("../data/prediction_mapping.csv", parse_dates=["forecast_start_date", "forecast_end_date"])
submission = sample_submission.merge(prediction_mapping, on="ID")

In [23]:
# For every forecast_start_date and forecast_end_date in submission make it 2024 instead of 2025
submission['forecast_start_date'] = submission['forecast_start_date'].apply(lambda x: x.replace(year=2024))
submission['forecast_end_date'] = submission['forecast_end_date'].apply(lambda x: x.replace(year=2024))

In [24]:
for rm_id, date_arrival, net_weight in orders_with_receivals_2024.itertuples(index=False):
    # Convert date_arrival to naive datetime for comparison
    date_arrival_naive = date_arrival.replace(tzinfo=None)
    submission.loc[
        (submission['rm_id'] == rm_id) & (submission['forecast_end_date'] >= date_arrival_naive),
        'predicted_weight'
    ] += net_weight

In [25]:
# save submission to a csv
submission.to_csv("./submission_2024receivals.csv", index=False)

In [26]:
test_submission = pd.read_csv("./submission_2024receivals.csv")
test_submission = test_submission[["ID", "predicted_weight"]]
test_submission["predicted_weight"] = test_submission["predicted_weight"] * 0.7
test_submission.to_csv("./submission_2024receivals_scale07.csv", index=False)